In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("SBAnational.csv")

/tmp/ipykernel_67977/3536806131.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SBAnational.csv")


### Name

In [3]:
df.drop(columns=["Name"], inplace=True)# variable non pertinente

### City

In [4]:
# suppression de la colonne City
# valeurs manquantes et pas de deduction fiable à partir de Zip ou State
df.drop(columns=["City"], inplace=True)

### State

Valeurs manquantes dans la colonne 'State'.
J'ai d'abord essayé de compléter ces valeurs avec ('Zip'). Mais, certains zip correspondent à plusieurs États.
Pour les cas ambigues j'ai utilisé la colonne 'BankState'.
ET BAM!!!!

In [5]:
zip_state_counts = df.groupby("Zip")["State"].nunique()

zip_state = df.groupby("Zip")["State"].first()

df["State"] = df.apply(lambda row: zip_state[row["Zip"]] if pd.isnull(row["State"]) and row["Zip"] in zip_state else row["State"], axis=1)

missing_state = df["State"].isnull().sum()
print(f" {missing_state}")


 1


In [6]:

if missing_state > 0:
    df["State"] = df.apply(lambda row: row["BankState"] if pd.isnull(row["State"]) else row["State"], axis=1)

final_missing_state = df["State"].isnull().sum()
print(f"{final_missing_state}")


0


### Zip

In [7]:
# Passer 'Zip' en string pour la traiter comme une donnée catégorielle
df["Zip"] = df["Zip"].astype(str).str.zfill(5)

In [8]:
df["Zip"].unique()

array(['47711', '46526', '47401', ..., '70036', '66549', '26134'],
      shape=(33611,), dtype=object)

Je suis en train de réfléchir à l'intégration de la colonne Zip dans le modèle ML. Elle pourrait être encodée par une variable catégorique ou utilisé uniquement les states, mais ça prend temps. (33611 zip différents, aie aie aie ouille ouille ouille)

### Bank

Ici nous avons une problématique avec les valeurs manquante, comment les remplacer.

In [9]:
df["Bank"] = df["Bank"].fillna("Unknown")

Réflexion, si on regarde la bank la plus populaire dans le secteur(State)... 
on pourrait peut être remplacer Unknown par une valeur plausible...?? GPTTTTTTT HELLLP

In [10]:

bank_mode_by_state = (
    df[df["Bank"] != "Unknown"]
    .groupby("BankState")["Bank"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
)

# Fonction d'imputation pour la colonne Bank
def impute_bank(row):
    # Si Bank vaut "Unknown" et que BankState est renseigné, on essaie d'utiliser le mapping
    if row["Bank"] == "Unknown":
        state = row["BankState"]
        if pd.notnull(state) and state in bank_mode_by_state.index:
            return bank_mode_by_state[state]
    return row["Bank"]

# Appliquer l'imputation ligne par ligne
df["Bank"] = df.apply(impute_bank, axis=1)

# Vérifier le résultat
print(df["Bank"].isnull().sum())
print(df["Bank"].value_counts().head())


0
Bank
BANK OF AMERICA NATL ASSOC        86853
WELLS FARGO BANK NATL ASSOC       63503
JPMORGAN CHASE BANK NATL ASSOC    48167
U.S. BANK NATIONAL ASSOCIATION    35143
CITIZENS BANK NATL ASSOC          35054
Name: count, dtype: int64


# BAAAM!!

### BankState

utile ou non... bof sinon on essaye de remplir les valeurs manquantes en cherchant directement avec la colonne bank.

In [11]:
# On crée d'abord un mapping : pour chaque banque, le mode (la valeur la plus fréquente) de BankState
bank_state_mode = (
    df[df["BankState"].notnull()]
    .groupby("Bank")["BankState"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
)

# Fonction modifiée pour imputer la valeur manquante dans BankState en se basant sur la banque
def fill_bankstate(row):
    if pd.isnull(row["BankState"]):
        # Si la banque est présente dans le mapping, on retourne sa valeur la plus fréquente
        if row["Bank"] in bank_state_mode:
            return bank_state_mode[row["Bank"]]
        else:
            return "Unknown"
    else:
        return row["BankState"]

df["BankState"] = df.apply(fill_bankstate, axis=1)

print(f"Valeurs manquantes dans 'BankState' après imputation : {df['BankState'].isnull().sum()}")


Valeurs manquantes dans 'BankState' après imputation : 0


In [12]:
df["BankState"].isin(["Unknown"]).sum()

np.int64(1566)

bofbof, le resultat n'est pas hyper concluant.

### NAICS

Je ne sais pas trop quoi en faire à par le passer en string comme pour zip.

In [13]:
df["NAICS"].describe()

count    899164.000000
mean     398660.950146
std      263318.312760
min           0.000000
25%      235210.000000
50%      445310.000000
75%      561730.000000
max      928120.000000
Name: NAICS, dtype: float64

### ApprovalDate

In [14]:
# Je la convertis en type datetime meme si on ne l'utilise pas
df["ApprovalDate"] = pd.to_datetime(df["ApprovalDate"], format="%d-%b-%y")

### ApprovalFY

In [15]:
#same here en entier


### Term

La colonne 'Term' => durée du prêt en mois.
RAS

### NoEmp

La colonne => nombre employées. RAS

### CreatedJob & RetainedJob

RAS 

### NewExist

In [16]:
df["NewExist"].value_counts()

NewExist
1.0    644869
2.0    253125
0.0      1034
Name: count, dtype: int64

pour choisir que faire avec les 0, on regarde les stats pour trouver un pattern. si on vois une ressemblance avec 1 ou 2 on pourra fusionner.

In [17]:
# On regroupe les données par la variable NewExist
# et on calcule quelques statistiques pour les colonnes NoEmp, CreateJob et RetainedJob.
grouped_stats = df.groupby("NewExist").agg({
    'NoEmp': ['count', 'mean', 'median', 'std'],
    'CreateJob': ['mean', 'median', 'std'],
    'RetainedJob': ['mean', 'median', 'std']
})

print(grouped_stats)


           NoEmp                                CreateJob                     \
           count       mean median         std       mean median         std   
NewExist                                                                       
0.0         1034  21.710832    8.0  161.383517  31.205996    0.0  483.032219   
1.0       644869  13.256068    5.0   75.792664   8.675672    0.0  244.754617   
2.0       253125   6.670799    2.0   68.819799   7.715516    0.0  213.146927   

         RetainedJob                     
                mean median         std  
NewExist                                 
0.0        28.193424    0.0  473.529201  
1.0        12.104979    1.0  245.211904  
2.0         7.395619    0.0  213.567956  


On observe bien que NewExist = 0 présentent des caractéristiques similaires à ceux avec NewExist = 1 . Je décide donc de fusionner les deux.
On remplace 0 par 1.

In [18]:
df["NewExist"] = df["NewExist"].replace(0, 1)

### FranchiseCode

In [19]:
df["FranchiseCode"].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930], shape=(2768,))

Ici on a beaucoup de valeurs différentes, on va les regrouper en deux catégories, franchise ou non, dans une nouvelle colonne.

In [20]:
df["IsFranchise"] = (df["FranchiseCode"] != 0).astype(int)
df["IsFranchise"].value_counts()

IsFranchise
1    690329
0    208835
Name: count, dtype: int64

### UrbanRural

je penses garder les 3 catégories ici... 
1 = Urban, 2 = rural, 0 = undefined

In [21]:
# Vérifions la distribution pour voir la proportion de chaque catégorie.
print(df['UrbanRural'].value_counts())


UrbanRural
1    470654
0    323167
2    105343
Name: count, dtype: int64


### RevLineCr

La colonne 'RevLineCr' indique si le prêt inclut une ligne de crédit renouvelable.
Je remplace les valeurs manquantes par 'N', je nclean la str,
et je mappe 'Y' à 1 et 'N' à 0 pour avoir une variable binaire.


In [22]:

df["RevLineCr"] = df["RevLineCr"].fillna("N").str.strip().str.upper()
df["RevLineCr"] = df["RevLineCr"].map({"Y": 1, "N": 0})


In [23]:
df["RevLineCr"].value_counts()

RevLineCr
0.0    424816
1.0    201397
Name: count, dtype: int64

### LowDoc

In [24]:
df["LowDoc"].value_counts()

LowDoc
N    782822
Y    110335
0      1491
C       758
S       603
A       497
R        75
1         1
Name: count, dtype: int64

Valeurs bizarre ici, plusieurs possibilité, soit Y et les autres=N
Soit Y et 1 et les autres = N 

In [25]:
#fonction de recodage

def recode_lowdoc(val):
    # On convertit en chaîne, on retire les espaces et on met en majuscules
    v = str(val).strip().upper()
    # Si la valeur est 'Y' ou '1', on la considère comme "oui" (1), sinon comme "non" (0)
    if v in ['Y']:
        return 1
    else:
        return 0

df["LowDoc"] = df["LowDoc"].apply(recode_lowdoc)

print(df["LowDoc"].value_counts())


LowDoc
0    788829
1    110335
Name: count, dtype: int64


### DisbursementGross

Clean clean clean


In [26]:

df["DisbursementGross"] = df["DisbursementGross"].str.replace('[\$,]', '', regex=True).str.strip().astype(float)


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_67977/529597651.py:1: SyntaxWarning: invalid escape sequence '\$'
  df["DisbursementGross"] = df["DisbursementGross"].str.replace('[\$,]', '', regex=True).str.strip().astype(float)


In [27]:

# Vérification rapide des premières lignes pour s'assurer que le nettoyage a fonctionné
print(df["DisbursementGross"].head())


0     60000.0
1     40000.0
2    287000.0
3     35000.0
4    229000.0
Name: DisbursementGross, dtype: float64


### GrAppv

clean clean borring


In [28]:
df["GrAppv"] = (df["GrAppv"].str.replace('[\$,]', '', regex=True).str.strip().astype(float))


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_67977/1115507837.py:1: SyntaxWarning: invalid escape sequence '\$'
  df["GrAppv"] = (df["GrAppv"].str.replace('[\$,]', '', regex=True).str.strip().astype(float))


### SBA_Appv

In [29]:
df["SBA_Appv"] = (df["SBA_Appv"].str.replace('[\$,]', '', regex=True).str.strip().astype(float))

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_67977/2503268751.py:1: SyntaxWarning: invalid escape sequence '\$'
  df["SBA_Appv"] = (df["SBA_Appv"].str.replace('[\$,]', '', regex=True).str.strip().astype(float))


### MIS_Status

On n'oublie pas la Target.

In [30]:
df['MIS_Status'] = df['MIS_Status'].fillna(
    df['ChgOffDate'].apply(lambda x: 'P I F' if pd.isna(x) else 'CHGOFF')
)

df['MIS_Status'] = df['MIS_Status'].map({'P I F': 1, 'CHGOFF': 0})

df['MIS_Status'].isna().sum()

np.int64(0)

## VOILAAA pour le moment on a nettoyer les données manquantes et maintenant je retire les colonnes qui leak la target:

reflexion sur zip grappv et sba_appv... pertinence à vérifier.

In [31]:
cols_to_drop = ["ChgOffDate", "BalanceGross", "ChgOffPrinGr", "DisbursementDate"]# "GrAppv", "SBA_Appv"
df.drop(columns=cols_to_drop, inplace=True)

In [32]:
print("Colonnes restantes après suppression du target leakage:")
print(df.columns)


Colonnes restantes après suppression du target leakage:
Index(['LoanNr_ChkDgt', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'DisbursementGross', 'MIS_Status', 'GrAppv', 'SBA_Appv', 'IsFranchise'],
      dtype='object')


In [33]:
ordre_colonnes =['MIS_Status', 'LoanNr_ChkDgt',
        'State', 'Zip', 'UrbanRural', 'Bank', 'BankState', #localisation
        'NAICS', 'NoEmp', 'NewExist', 'CreateJob','RetainedJob','FranchiseCode', 'IsFranchise', #entreprise
        'Term','RevLineCr', 'LowDoc','DisbursementGross','ApprovalDate', 'ApprovalFY', 'GrAppv', 'SBA_Appv'] #prêt

df = df[ordre_colonnes]
df

,MIS_Status,LoanNr_ChkDgt,State,Zip,UrbanRural,Bank,BankState,NAICS,NoEmp,NewExist,...,FranchiseCode,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,ApprovalDate,ApprovalFY,GrAppv,SBA_Appv
0,1,1000014003,IN,47711,0,FIFTH THIRD BANK,OH,451120,4,2.0,...,1,1,84,0.0,1,60000.0,1997-02-28,1997,60000.0,48000.0
1,1,1000024006,IN,46526,0,1ST SOURCE BANK,IN,722410,2,2.0,...,1,1,60,0.0,1,40000.0,1997-02-28,1997,40000.0,32000.0
2,1,1000034009,IN,47401,0,GRANT COUNTY STATE BANK,IN,621210,7,1.0,...,1,1,180,0.0,0,287000.0,1997-02-28,1997,287000.0,215250.0
3,1,1000044001,OK,74012,0,1ST NATL BK & TR CO OF BROKEN,OK,0,2,1.0,...,1,1,60,0.0,1,35000.0,1997-02-28,1997,35000.0,28000.0
4,1,1000054004,FL,32801,0,FLORIDA BUS. DEVEL CORP,FL,0,14,1.0,...,1,1,240,0.0,0,229000.0,1997-02-28,1997,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,1,9995573004,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,6,1.0,...,1,1,60,NaN,0,70000.0,1997-02-27,1997,70000.0,56000.0
899160,1,9995603000,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,6,1.0,...,1,1,60,1.0,0,85000.0,1997-02-27,1997,85000.0,42500.0
899161,1,9995613003,CA,93455,0,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,26,1.0,...,1,1,108,0.0,0,300000.0,1997-02-27,1997,300000.0,225000.0
899162,0,9995973006,HI,96830,0,BANK OF HAWAII,HI,0,6,1.0,...,1,1,60,0.0,1,75000.0,1997-02-27,1997,75000.0,60000.0


# DONE

In [34]:
df.isna().sum()

MIS_Status                0
LoanNr_ChkDgt             0
State                     0
Zip                       0
UrbanRural                0
Bank                      0
BankState                 0
NAICS                     0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
IsFranchise               0
Term                      0
RevLineCr            272951
LowDoc                    0
DisbursementGross         0
ApprovalDate              0
ApprovalFY                0
GrAppv                    0
SBA_Appv                  0
dtype: int64

oups je me rends compte que j'ai oublier des valeurs nulles dans la colonne NewExist... comment gerer ça?

In [35]:
df['NewExist'].unique()

array([ 2.,  1., nan])

In [36]:
# Imputation des valeurs manquantes dans NewExist en se basant sur l'analyse statistique
# On impute avec 2 (entreprises nouvelles) car les caractéristiques des enregistrements manquants se rapprochent davantage du groupe 2.
df["NewExist"] = df["NewExist"].fillna(2)

# Vérification rapide de la distribution après imputation
print(df["NewExist"].value_counts())



NewExist
1.0    645903
2.0    253261
Name: count, dtype: int64


In [37]:
df.isna().sum()

MIS_Status                0
LoanNr_ChkDgt             0
State                     0
Zip                       0
UrbanRural                0
Bank                      0
BankState                 0
NAICS                     0
NoEmp                     0
NewExist                  0
CreateJob                 0
RetainedJob               0
FranchiseCode             0
IsFranchise               0
Term                      0
RevLineCr            272951
LowDoc                    0
DisbursementGross         0
ApprovalDate              0
ApprovalFY                0
GrAppv                    0
SBA_Appv                  0
dtype: int64

pour RevLineCr aussi on a un soucis, les valeur nulles nont pas été changer. car nan.

In [38]:
df["RevLineCr"].unique()

array([ 0., nan,  1.])

In [39]:
df["RevLineCr"] = df["RevLineCr"].astype(str)
df["RevLineCr"] = df["RevLineCr"].replace("nan", "N")

df["RevLineCr"] = df["RevLineCr"].map(lambda x: 1 if x == "Y" else 0)

print( df["RevLineCr"].isnull().sum())


0


In [40]:
df["RevLineCr"].value_counts()

RevLineCr
0    899164
Name: count, dtype: int64

In [41]:
df.isna().sum()

MIS_Status           0
LoanNr_ChkDgt        0
State                0
Zip                  0
UrbanRural           0
Bank                 0
BankState            0
NAICS                0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
IsFranchise          0
Term                 0
RevLineCr            0
LowDoc               0
DisbursementGross    0
ApprovalDate         0
ApprovalFY           0
GrAppv               0
SBA_Appv             0
dtype: int64

voila plus de valeur nulle, parfaitement nettoyé... 

In [42]:
df.to_csv("cleaned_data.csv", index=False)

In [43]:
df

,MIS_Status,LoanNr_ChkDgt,State,Zip,UrbanRural,Bank,BankState,NAICS,NoEmp,NewExist,...,FranchiseCode,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,ApprovalDate,ApprovalFY,GrAppv,SBA_Appv
0,1,1000014003,IN,47711,0,FIFTH THIRD BANK,OH,451120,4,2.0,...,1,1,84,0,1,60000.0,1997-02-28,1997,60000.0,48000.0
1,1,1000024006,IN,46526,0,1ST SOURCE BANK,IN,722410,2,2.0,...,1,1,60,0,1,40000.0,1997-02-28,1997,40000.0,32000.0
2,1,1000034009,IN,47401,0,GRANT COUNTY STATE BANK,IN,621210,7,1.0,...,1,1,180,0,0,287000.0,1997-02-28,1997,287000.0,215250.0
3,1,1000044001,OK,74012,0,1ST NATL BK & TR CO OF BROKEN,OK,0,2,1.0,...,1,1,60,0,1,35000.0,1997-02-28,1997,35000.0,28000.0
4,1,1000054004,FL,32801,0,FLORIDA BUS. DEVEL CORP,FL,0,14,1.0,...,1,1,240,0,0,229000.0,1997-02-28,1997,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,1,9995573004,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,6,1.0,...,1,1,60,0,0,70000.0,1997-02-27,1997,70000.0,56000.0
899160,1,9995603000,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,6,1.0,...,1,1,60,0,0,85000.0,1997-02-27,1997,85000.0,42500.0
899161,1,9995613003,CA,93455,0,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,26,1.0,...,1,1,108,0,0,300000.0,1997-02-27,1997,300000.0,225000.0
899162,0,9995973006,HI,96830,0,BANK OF HAWAII,HI,0,6,1.0,...,1,1,60,0,1,75000.0,1997-02-27,1997,75000.0,60000.0
